# **Statistical Analyst Time Series Using Sunspot Data**
---
pada tahap ini kita akan melakukan preprocesing data lebih lanjut dengan menggunakan statistical analyst 

## **1.Statistical Analyst Using Sunspot Data**

### **1.1 Import Library**
---
pertama kita akan melakukan import library yang dibutuhkan untuk proses analisis statistik pada data sunspot

In [ ]:
# import library yang dibutuhkan 
import numpy as np                              # import library untuk komputasi numerik 
import pandas as pd                             # import library untuk pengolahan datafrane 
import matplotlib.pyplot as plt                 # import library untuk visualisasi data dengan matplotlib 
import seaborn as sns                           # import library untuk visualisasi data dengan seaborn 
import os                                       # import library os untuk 

from warnings import simplefilter               # import library untuk handling error 
from scipy import stats
from statsmodels.tsa.arima_model import ARIMA   # import library untuk pengolahan statistik dengan model ARIMA (Autoregresive moving average)
from statsmodels.graphics.api import qqplot     # import library untuk pengolahan statistik dalam bentuk visualisasi data

# definisikan variabel untuk memfilter error yang ada 
simplefilter('ignore')

### **1.2 Load File CSV dengan library OS dan membaca file tersebut dengan pandas**
---
pada tahap ini kita akan melakukan load file yang sudah diolah sebelum nya pada kode exploration.ipynb dan menggunakan file tersebut untuk analisis lebih lanjut 

In [16]:
# Path ke folder
folder_path = "../data/processed"

# Menelusuri semua file di folder dan subfolder
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith(".csv"):  # Menemukan file CSV
            file_path = os.path.join(root, file)
            print(f"File ditemukan: {file_path}")
            
            # Membaca file ke pandas
            df = pd.read_csv(file_path)
            
            # Menampilkan 5 baris pertama
            print(df.head())


File ditemukan: ../data/processed\processed_file.csv
         Date  Monthly Mean Total Sunspot Number  Month  nth_year
0  1749-01-31                               96.7      1         9
1  1749-02-28                              104.3      2         9
2  1749-03-31                              116.7      3         9
3  1749-04-30                               92.8      4         9
4  1749-05-31                              141.7      5         9
